In [37]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re
import gensim
from gensim.models import KeyedVectors
import numpy as np
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases
from nltk import sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [38]:
custom_stopwords = ['echo', 'alexa', "alexia", 'dot', "star", 'amazon', 'prime', '2nd', 'generation', "fire", "stick", "firestick", "skype", "facetime", '1st', '3rd', '4th', '5th', "hub", "hulu", 'google', 'netflix', "sony", 'youtube', 'philip', 'tp-link', 'fourth', 'roku', "siri", 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "...", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 'wasn', 'weren', 'won', 'wouldn']

def negation_handler(sentence):	

    """Handle negations using WordNet. See https://github.com/UtkarshRedd/Negation_handling for a clear explenation."""

    temp = int(0)
    for i in range(len(sentence)):
        if sentence[i-1] in ['not',"n't", "no", "without"]:
            antonyms = []
            for syn in wordnet.synsets(sentence[i]):
                syns = wordnet.synsets(sentence[i])
                w1 = syns[0].name()
                temp = 0
                for l in syn.lemmas():
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                max_dissimilarity = 0
                for ant in antonyms:
                    syns = wordnet.synsets(ant)
                    w2 = syns[0].name()
                    syns = wordnet.synsets(sentence[i])
                    w1 = syns[0].name()
                    word1 = wordnet.synset(w1)
                    word2 = wordnet.synset(w2)
                    if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                        temp = 1 - word1.wup_similarity(word2)
                    if temp>max_dissimilarity:
                        max_dissimilarity = temp
                        antonym_max = ant
                        sentence[i] = antonym_max
                        sentence[i-1] = ''
    while '' in sentence:
        sentence.remove('')
    return sentence

In [39]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t")
dataset.drop(dataset[dataset.rating == 3].index, inplace=True) #droppa recensioni con 3-4 stelle
dataset.drop(dataset[dataset.rating == 4].index, inplace=True) #droppa recensioni con 3-4 stelle
dataset.tail()

,rating,date,variation,verified_reviews,feedback
3144,5,30-Jul-18,Black Dot,love it,1
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


In [40]:
#  CREAZIONE DI UN SAMPLE DATASET BILANCIATO
# prende le prime n recensioni positive di lunghezza maggiore, dove n è il numero di recensioni negative
def create_balanced_dataset(dataset):
    """Bilancia il dataset uniformando il numero di recensioni negative e positive. Prende in input il dataset"""
    reviews_1 = list(dataset[dataset["feedback"] == 1]["verified_reviews"])
    reviews_0 = list(dataset[dataset["feedback"] == 0]["verified_reviews"])
    reviews_1.sort(key=len, reverse = True)
    sample_1 = reviews_1[:len(reviews_0)]
    verified_reviews_sample = []
    feedback_sample = []
    verified_reviews_sample.extend(sample_1)
    verified_reviews_sample.extend(reviews_0)
    feedback_sample.extend([1 for i in range(len(sample_1))])
    feedback_sample.extend([0 for i in range(len(reviews_0))])
    dataset = pd.DataFrame({"verified_reviews":verified_reviews_sample, "feedback": feedback_sample})
    print(dataset["feedback"].value_counts())
    return dataset


# dataset = create_balanced_dataset(dataset)

1    257
0    257
Name: feedback, dtype: int64


In [41]:
def get_wordnet_pos(treebank_tag):
    """
    return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v).
    This was done in order to have compatibility with the wordnet lemmatizer.
    For example the pos "JJ" is transformed in "a".
    """
    if treebank_tag.startswith('J'):
        return "a"
    elif treebank_tag.startswith('V'):
        return "v"
    elif treebank_tag.startswith('N'):
        return "n"
    elif treebank_tag.startswith('R'):
        return "r"
    else:
        return "n"
        
pos_list = ["JJ", "JJR", "JJS", "RB", "RBR", "RBS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]

def tokenize_list_of_text(list_of_text, custom_stopwords = [], pos_filter = False, pos_list = []):
    """Tokenizza tutte le recensioni, pulisce da stopwords, elimina token <= 2 caratteri e lemmatizza.
    Ritorna sia il la lista tokenizzata ma come stringa sia come lista di tokens, dunque ritorna due elementi"""

    tokenizer = nltk.tokenize.TweetTokenizer()
    lemmatizer = nltk.WordNetLemmatizer()
    detokenizer = TreebankWordDetokenizer()

    tokenized_reviews = []
    sent_tokenized_reviews = []
    pos_reviews = []
    for review in list_of_text: #pulisce le recensioni
        review = re.sub(r'\d+', '', review) # elimina i numeri
        clean_text = ""
        tokens = nltk.tokenize.word_tokenize(review, language='english', preserve_line=False)
        tokens = negation_handler(tokens)
        tokens = [w.lower() for w in tokens]
        tokens_pos = pos_tag(tokens)
        lemmatized_tokens = [(lemmatizer.lemmatize(w, get_wordnet_pos(pos)), pos) for w, pos in tokens_pos]
        if pos_filter:
            clean_tokens = [w for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>2 and w not in custom_stopwords and pos in pos_list]
        else:
            clean_tokens = [w for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>2 and w not in custom_stopwords]
        sent_tokenized_reviews.append(clean_tokens)
        tokenized_reviews.append(detokenizer.detokenize(clean_tokens))
    
    n_tokens = []
    for sent in sent_tokenized_reviews:
        for w in sent:
            n_tokens.append(w)
    print("total number of tokens extracted are:", len(set(n_tokens)))
    return tokenized_reviews,  sent_tokenized_reviews # ritorna una tupla!

In [42]:
new_text, new_sent_tok = tokenize_list_of_text(dataset["verified_reviews"], custom_stopwords, False, pos_list)
# contengono una lista di tutte le frasi pre processate, nella prima variabile in stringa, nella seconda in tokens 

total number of tokens extracted are: 2154


In [43]:
# Elimino token con freq minore di 5
tot_tokens = []

for sent in new_sent_tok:
    for tok in sent:
        tot_tokens.append(tok)

freqs = nltk.FreqDist(tot_tokens)
cleaned_reviews = []

for sent in new_sent_tok:
    clean_sent = []
    for tok in sent:
        if freqs[tok] > 4:
            clean_sent.append(tok)
    cleaned_reviews.append(clean_sent)

cleaned_reviews

[['piece',
  'right',
  'living',
  'room',
  'kitchen',
  'counter',
  'speaker',
  'every',
  'direction',
  'quality',
  'sound',
  'quite',
  'good',
  'connect',
  'via',
  'bluetooth',
  'find',
  'sound',
  'equipment',
  'good',
  'bass',
  'sound',
  'loud',
  'enough',
  'bother',
  'top',
  'work',
  'great',
  'adjust',
  'volume',
  'first',
  'device',
  'would',
  'imagine',
  'volume',
  'button',
  'large',
  'not',
  'alone',
  'would',
  'recommend',
  'regular',
  'piece',
  'look',
  'quality',
  'quite',
  'good',
  'counter',
  'even',
  'without',
  'order',
  'come',
  'free',
  'hue',
  'bulb',
  'along',
  'extra',
  'one',
  'buy',
  'put',
  'bulb',
  'living',
  'room',
  'floor',
  'lamp',
  'turn',
  'light',
  'say',
  'connect',
  'device',
  'default',
  'name',
  'bulb',
  'first',
  'light',
  'second',
  'light',
  'dim',
  'floor',
  'lamp',
  'turn',
  'one',
  'light',
  'say',
  'turn',
  'second',
  'light',
  'app',
  'create',
  'first',
  '

In [44]:
bigrams = Phrases(cleaned_reviews) #estrae le collocazioni tramite PMI
bigrams[cleaned_reviews][12]

['never',
 'think',
 'person',
 'completely',
 'connect',
 'purchase',
 'another',
 'product',
 'day',
 'love',
 'one',
 'office',
 'one',
 'kitchen',
 'already',
 'use',
 'morning',
 'news',
 'get',
 'ready',
 'day',
 'well',
 'look',
 'recipe',
 'screen',
 'big',
 'enough',
 'easily',
 'read',
 'love',
 'much',
 'purchase',
 'parent',
 'easily',
 'video',
 'call',
 'issue',
 'computer',
 'video',
 'tool',
 'purchase',
 'camera',
 'cover',
 'even',
 'though',
 'button',
 'turn',
 'camera',
 'free',
 'camera',
 'work',
 'around']

In [45]:
X_train, X_test, Y_train, Y_test = train_test_split(bigrams[cleaned_reviews], dataset["feedback"].values, test_size=0.20, random_state=10)

In [46]:
# Questa parte estrare le parole più rilevanti per ogni categoria, ma non la uso alla fine.
# Lo riprenderò più avanti
# FARE IL K-SQUARED

neg_tokenized = [rev for rev, feedback in zip(X_train, Y_train) if feedback == 0]
pos_tokenized = [rev for rev, feedback in zip(X_train, Y_train) if feedback == 1]
neg_tokenized = [w for rev in neg_tokenized for w in rev]
pos_tokenized = [w for rev in pos_tokenized for w in rev]

import math 

def get_pos_neg_score(pos, neg):
    total_tokens = pos + neg
    total_tokens_freq = nltk.FreqDist(total_tokens)
    pos_freq = nltk.FreqDist(pos)
    neg_freq = nltk.FreqDist(neg)
    data_dict = {"pos": {}, "neg": {}}

    for w in pos:
        p_category = pos_freq[w] / total_tokens_freq[w]
        p = pos_freq[w] / len(pos)
        data_dict["pos"][w] = p_category * p
    
    for w in neg:
        p_category = neg_freq[w] / total_tokens_freq[w]
        p = neg_freq[w] / len(neg)
        data_dict["neg"][w] = p_category * p

    risultato = {}

    for label in ["pos", "neg"]:
        values_list = list(data_dict[label].values())
        min_values = min(values_list)
        max_values = max(values_list)
        for k in data_dict[label]:
            val = data_dict[label][k]
            data_dict[label][k] = (val- min_values)/(max_values-min_values)

    return data_dict

data_dict = get_pos_neg_score(pos_tokenized, neg_tokenized)

res = {}

for w in pos_tokenized+neg_tokenized:
    res[w] = max(data_dict["pos"].get(w, 0.01), data_dict["neg"].get(w, 0.01))

{k: v for k, v in sorted(res.items(), key = lambda item: item[1], reverse=True)}

{'love': 1.0,
 'work': 1.0,
 'not': 0.9515674345983887,
 'device': 0.7319601145164591,
 'would': 0.6357908484240368,
 'try': 0.5453517447879338,
 'product': 0.48963649245246466,
 'use': 0.4868580507449486,
 'get': 0.4862738214370634,
 'stop': 0.4463503591507796,
 'great': 0.443300701456322,
 'buy': 0.4295857140823497,
 'time': 0.42880957310695944,
 'want': 0.4144999844922641,
 'return': 0.4120314299346097,
 'like': 0.40358514491956066,
 'one': 0.40351159483777393,
 'money': 0.3472965085503837,
 'back': 0.34469353094227667,
 'thing': 0.3105045976701075,
 'disappointed': 0.2954995991739266,
 'need': 0.2667259636058909,
 'unplug': 0.2643647039461459,
 'music': 0.26408895226187057,
 'terrible': 0.2621721056906684,
 'good': 0.25949308876727556,
 'sound': 0.25597851848894076,
 'day': 0.25484279773880875,
 'light': 0.2533116284707058,
 'month': 0.25164763406437635,
 'purchase': 0.24885729969350923,
 'speaker': 0.2408652144685929,
 'half': 0.23825285199455007,
 'screen': 0.2365187061015815,
 '

In [47]:
# tfidf weighting sui token, per ora senza pos. Mi sono reso conto che la pos per averla più accurata la devo fare prima del pre processing.

tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False, min_df = 0)
tfidf_model = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)
review = 10
for score, feature in zip(tfidf_model.toarray()[review], tfidf.get_feature_names_out()):
    if score > 0.0:
        print(feature, score)

alarm_clock 0.20638304798026263
also 0.15128443650138138
ask 0.16805632372808949
control 0.19837381356374303
easy_set 0.21595141664870182
give 0.18938866892822193
home 0.16180976447314752
look 0.19367675104498014
love 0.22348662699513341
n't 0.15421293292855337
news 0.2125513199183368
nightstand 0.26669297209389115
scroll 0.26669297209389115
smart 0.17839174065326277
smart_plug 0.22783633031005418
speaker 0.146759990115679
spot 0.19596934373822156
sure 0.20638304798026263
sync 0.2435303211037762
technology 0.2435303211037762
use 0.2352020282694477
weather 0.16805632372808949
wi-fi 0.26669297209389115


In [48]:
w2v_model = Word2Vec(X_train, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007)

In [49]:
w2v_model.train(X_train, total_examples=len(X_train), epochs=100)

(662303, 854500)

In [50]:
w2v_model.wv.most_similar("love", topn = 10)

[('lyric', 0.24185948073863983),
 ('one', 0.22020143270492554),
 ('front', 0.21854150295257568),
 ('absolutely', 0.21308615803718567),
 ('office', 0.19493290781974792),
 ('much', 0.19368916749954224),
 ('tap', 0.19235697388648987),
 ('version', 0.19198137521743774),
 ('wall', 0.1914304494857788),
 ('chat', 0.19028235971927643)]

In [51]:
w2v_model.wv.most_similar("love", topn = 2000)[-10:]

[('pick', -0.03731299191713333),
 ('step', -0.03874468803405762),
 ('bass', -0.0423424132168293),
 ('phone', -0.045248303562402725),
 ('suck', -0.054290302097797394),
 ('direction', -0.0572613961994648),
 ('view', -0.057648662477731705),
 ('question', -0.071042999625206),
 ('equipment', -0.08263342827558517),
 ('number', -0.09720328450202942)]

In [52]:
df = pd.DataFrame(tfidf_model.toarray(), columns = tfidf.get_feature_names_out())
df

,'ll,'re,'ve,....,ability,able,absolutely,access,account,act,...,worry,worth,worth_money,would,wrong,yard,year,year_old,yes,yet
0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.000000
1,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.000000
2,0.0,0.0,0.392436,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.000000
3,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.2772,0.0,0.000000
4,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.000000
407,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.136924,0.0000,0.0,0.000000
408,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.000000
409,0.0,0.0,0.000000,0.0,0.0,0.334091,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.266588,0.0,0.0,0.000000,0.0000,0.0,0.198434


In [53]:
g_model = []
# google_model = api.load('word2vec-google-news-300', True)
# g_model = KeyedVectors.load_word2vec_format(google_model, binary=True)

In [54]:
def review_vectors(tokens, size = 300, weights = [], pretrained = False, test = False):

    """Genera un vettore per ogni recensione: questo vettore
    è calcolato come la media ponderata (t * w: token vettore * perso tfidf) dei vettori dei token nella recensione.
    Il vettore risultante è normalizzato alla fine."""

    vec = np.zeros(size).reshape((1, size))
    count = 0

    if test:
        for word in tokens:
            try:
                if pretrained:
                    vec += g_model[word] * res[word]
                else:
                    vec += w2v_model.wv[word] * res[word]
                count +=1
            except KeyError:
                print("non trovo", word)
                continue

        if count!= 0:
            vec = vec / norm(vec)
        
        return vec

    if len(tokens) != len(weights):
        print("nope")

    for word, weight in zip(tokens, weights):
        try:
            if pretrained:
                vec += g_model[word] * res[word] * weight
            else:
                vec += w2v_model.wv[word] * res[word] * weight
            count +=1
        except KeyError:
            print("non trovo", word)
            continue

    if count!= 0:
        vec = vec / norm(vec)
        
    return vec

In [55]:
# qui viene creato per ogni recensione il vettore

w2v_X_train = np.zeros((len(X_train), 300))
for i in range(len(X_train)):
    w2v_X_train[i,:] = review_vectors(tfidf.inverse_transform(tfidf_model[i, :])[0], 300, tfidf_model[i,:].data, False)
w2v_df = pd.DataFrame(w2v_X_train)
w2v_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.043361,-0.030495,-0.003502,0.013358,-0.019625,-0.103967,0.089869,0.096184,0.009696,-0.048678,...,-0.015580,0.074518,0.040024,-0.035752,0.082640,-0.015861,-0.035971,-0.020993,0.044203,-0.131662
1,-0.061444,0.038947,-0.037872,0.030543,0.035012,-0.109029,0.024659,0.119979,-0.012062,-0.042627,...,-0.017490,0.138064,0.093377,-0.022957,0.067168,-0.020714,0.032577,-0.035973,0.024849,-0.046410
2,0.032147,0.084094,-0.084506,0.012356,0.097796,-0.193742,0.036152,0.043033,-0.001593,0.013833,...,-0.052919,0.113622,0.003633,-0.000624,-0.049663,0.046681,0.107524,-0.003601,0.000511,-0.044132
3,-0.091840,0.055676,-0.039934,0.062388,0.005016,-0.107825,0.080682,0.075926,-0.005628,-0.003238,...,-0.097288,0.071874,0.083020,-0.024480,0.056050,-0.024864,0.013586,-0.034006,-0.035509,-0.034419
4,0.006319,0.135592,0.030696,0.063624,-0.016375,-0.014690,0.110159,0.120721,0.071172,-0.010606,...,-0.034135,0.079319,0.047166,0.028069,0.055989,0.037249,-0.017724,-0.038220,0.034461,-0.068278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,0.083387,0.095299,-0.094947,-0.098256,0.011375,0.090161,0.033084,0.161004,0.027425,0.041525,...,-0.002966,-0.010106,0.105447,-0.085238,0.120982,0.088034,-0.060704,0.044460,0.015488,-0.073700
407,-0.020742,0.099221,0.000760,0.100361,0.008287,-0.079799,0.109182,0.084502,0.010657,0.001314,...,-0.075826,0.113319,0.032915,-0.033279,0.027193,0.026264,-0.009778,-0.032299,0.027542,-0.071942
408,-0.012633,0.097789,-0.026334,-0.050746,0.024037,-0.005642,0.075401,0.123181,-0.031785,-0.007981,...,0.008566,0.080144,0.110795,0.060669,0.032499,-0.013996,0.017020,0.044885,0.022279,0.005330
409,0.053227,0.009160,-0.077213,-0.004636,-0.036167,-0.044682,0.078622,0.164087,-0.096339,-0.029411,...,-0.025754,0.145672,0.077457,0.042844,0.094635,0.091272,0.033996,-0.006951,-0.017958,-0.026569


In [56]:
X_test_vec = np.zeros((len(X_test), 300))
for i in range(len(X_test)):
    X_test_vec[i,:] = review_vectors(tfidf.inverse_transform(X_test_tf[i, :])[0], 300, X_test_tf[i,:].data, False)

X_test_vec[0]

array([-2.43358086e-02,  1.03916534e-01,  6.21550324e-03,  1.53707320e-02,
        1.36603382e-02, -7.98567325e-03, -1.13990788e-02,  1.18108018e-01,
       -6.56364551e-02, -6.45404395e-02,  1.34079215e-03, -7.21432419e-02,
       -1.72550741e-02,  7.16500264e-02, -6.14462158e-02, -1.35974124e-02,
        1.19889835e-02,  6.12751202e-02,  5.11644961e-02,  1.55173357e-01,
       -5.89987936e-02, -5.54947940e-03, -1.37287411e-01,  5.26044484e-02,
        7.45619354e-02, -4.13337613e-02,  2.75840985e-03, -5.49506200e-02,
       -1.08327669e-01, -6.79422529e-02, -1.60668058e-02, -1.11493730e-02,
       -7.27450290e-02, -6.73163676e-03,  2.09592454e-03,  4.41556638e-02,
       -1.85037414e-02, -3.16491260e-02,  4.04708082e-02, -7.64954893e-03,
       -8.70169260e-02, -7.39565690e-03, -1.40254261e-02, -5.83034604e-02,
        1.82400002e-02,  6.07864149e-02,  1.06315604e-01,  1.18698419e-02,
        5.13241552e-02,  3.19051020e-03, -1.09200565e-01, -1.58706535e-01,
       -4.48160999e-02, -

In [61]:
svm = SVC(max_iter=3000)
svm_model = svm.fit(w2v_X_train, Y_train)
predictions = svm_model.predict(X_test_vec)
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      0.88      0.92        56
           1       0.87      0.96      0.91        47

    accuracy                           0.91       103
   macro avg       0.91      0.92      0.91       103
weighted avg       0.92      0.91      0.91       103



In [62]:
# risultati migliori:

# w2v + tf-idf weights + svm and word count > 4

# w2v configuration:
# w2v_model = Word2Vec(cleaned_reviews, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007) and 100 epochs for training

# otherwise the balanced dataset performs better

In [63]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [64]:
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf','linear', 'poly'],
              'class_weight':['balanced', None]
}

In [65]:
grid = GridSearchCV(SVC(), param_grid, refit ='f1_macro', scoring="f1_macro", cv=kfold, verbose=3)
grid.fit(w2v_X_train, Y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.891 total time=   0.0s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.914 total time=   0.0s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.864 total time=   0.0s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.902 total time=   0.0s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.829 total time=   0.0s
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.916 total time=   0.0s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.902 total time=   0.0s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.939 total time=   0.0s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.878 total time=   0.0s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=lin

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'class_weight': ['balanced', None],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear', 'poly']},
             refit='f1_macro', scoring='f1_macro', verbose=3)

In [66]:
print(grid.best_params_)

{'C': 1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'}


In [67]:
grid_predictions = grid.predict(X_test_vec)

print(classification_report(Y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.96      0.88      0.92        56
           1       0.87      0.96      0.91        47

    accuracy                           0.91       103
   macro avg       0.91      0.92      0.91       103
weighted avg       0.92      0.91      0.91       103

